In [1]:
import os
from dotenv import load_dotenv
from google import genai 
from google.genai import types 

import json 
import time 
import uuid 
from pathlib import Path 
from typing import Any , Dict , List , Optional , Tuple 



In [2]:
# --- Load Env --- 
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")



In [3]:
# --- Get Client --- 
client = genai.Client()


In [4]:
# --- Utility helpers (polling, timing, safe JSON) ---
def now_ms() -> int:
    return int(time.time() * 1000)

def poll_operation(op, *, poll_s: float = 2.0, timeout_s: float = 600.0):
    """
        Poll a long-running operation until done or timeout.
    """
    start = time.time()
    while True:
        if getattr(op, "done", False):
            return op
        if time.time() - start > timeout_s:
            raise TimeoutError("Operation did not complete within timeout.")
        time.sleep(poll_s)
        op = client.operations.get(op)
        
def pretty(obj: Any) -> str:
    try:
        return json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    except Exception:
        return str(obj)

def unique_store_display_name(prefix: str = "warm-file-search") -> str:
    return f"{prefix}-{uuid.uuid4().hex[:10]}"

def ensure_file_exists(path: str) -> Path:
    p = Path(path).expanduser().resolve()
    if not p.exists():
        raise FileNotFoundError(f"File not found: {p}")
    return p

def file_size_mb(path: Path) -> float:
    return path.stat().st_size / (1024 * 1024)

def timed(fn, *args, **kwargs):
    t0 = now_ms()
    out = fn(*args, **kwargs)
    t1 = now_ms()
    return out, (t1 - t0)

In [5]:
# --- Let's Create a small sample file for Our Rag System --- 
sample_text = """Warm AI Internal Memo (Sample)

Product: Warm AI
Vision: Professional networking + research with grounded answers.

Key modules:
- Chat mode
- People search
- Company search
- File Search (RAG) prototype

Roadmap v2:
- Add File Search so users can upload a document and chat grounded in it.
- Extract citations for UI trust.

Pricing note (example):
- Indexing creates embeddings.
- Queries retrieve relevant chunks as context.
"""

Path("sample.txt").write_text(sample_text, encoding="utf-8")
FILE_PATH = "sample.txt"

p = ensure_file_exists(FILE_PATH)
print("Using file:", p)
print("Size MB:", round(file_size_mb(p), 4))

Using file: C:\Users\PCS\Desktop\SaaS\WarmAI\notebooks\sample.txt
Size MB: 0.0004


In [6]:
# --- Let's Start the Process --- 
# --- Step 1 : Create a File Search Store --- 
store_display_name = unique_store_display_name()
store = client.file_search_stores.create(config={"display_name" : store_display_name})
# --- Some File Store Metadata --- 
print("---- Store created :  ----")
print("display_name:" , store_display_name)
print("name:" , store.name)


---- Store created :  ----
display_name: warm-file-search-78a797be32
name: fileSearchStores/warmfilesearch78a797be32-qegtere0l1st


In [7]:
# --- Let's Start the Upload Processing --- 
# --- Step 2 : Upload Files to the File Search Store --- 
p = ensure_file_exists(FILE_PATH)

# --- Just To Show the Custimsation here , we will Config the Chunk --- 
chunking_config = {
    "white_space_config": {
        "max_tokens_per_chunk": 300,
        "max_overlap_tokens": 30
    }
}
# --- Start the Upload in Try/Catch --- 
try:
    op = client.file_search_stores.upload_to_file_search_store(
        file=str(p),
        file_search_store_name=store.name,
        config={
            "display_name": p.name,
            "chunking_config": chunking_config
        },
    )
except TypeError as e:
    print("TypeError on upload_to_file_search_store(file=...). Trying fallback signature.")
    op = client.file_search_stores.upload_to_file_search_store(
        file_search_store_name=store.name,
        file_name=str(p),
        config={
            "display_name": p.name,
            "chunking_config": chunking_config,
        },
    )

op = poll_operation(op, poll_s=2.0, timeout_s=900.0)
print("Indexing done. Operation:")
print(pretty(op))

Indexing done. Operation:
"name='fileSearchStores/warmfilesearch78a797be32-qegtere0l1st/upload/operations/sampletxt-cw1gumt1r1lf' metadata=None done=True error=None response=UploadToFileSearchStoreResponse(\n  document_name='fileSearchStores/warmfilesearch78a797be32-qegtere0l1st/documents/sampletxt-cw1gumt1r1lf',\n  parent='fileSearchStores/warmfilesearch78a797be32-qegtere0l1st'\n)"


In [8]:
# --- Let's Query the File Search Store --- 
MODEL_FLASH = "gemini-flash-latest"

query = "What is Warm AI and what is planned in v2?"

resp, ms = timed(
    client.models.generate_content,
    model=MODEL_FLASH,
    contents=query,
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[store.name]
                )
            )
        ],
        system_instruction=(
            "Answer ONLY using the uploaded document context. "
            "If the document does not contain the answer, say you cannot find it in the file."
        ),
        temperature=0.2,
    ),
)
print(f"Latency: {ms} ms\n")
print(resp.text)

Latency: 2058 ms

Warm AI is a product with a vision to provide professional networking and research capabilities with grounded answers.

Its key modules include:
*   Chat mode
*   People search
*   Company search
*   File Search (RAG) prototype

The plans for **Roadmap v2** are to:
1.  Add **File Search** so users can upload a document and chat grounded in it.
2.  Extract **citations** for UI trust.


In [9]:
# --- Grounding metdata --- 
gm = None 
try:
    gm = resp.candidates[0].grounding_metadata 
except Exception:
    gm = None 

print("grounding_metadata present:" , gm is not None)
print(pretty(gm))

grounding_metadata present: True
"google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(\n  retrieved_context=GroundingChunkRetrievedContext(\n    text=\"\"\"Warm AI Internal Memo (Sample)\r\n\r\nProduct: Warm AI\r\nVision: Professional networking + research with grounded answers.\r\n\r\nKey modules:\r\n- Chat mode\r\n- People search\r\n- Company search\r\n- File Search (RAG) prototype\r\n\r\nRoadmap v2:\r\n- Add File Search so users can upload a document and chat grounded in it.\r\n- Extract citations for UI trust.\r\n\r\nPricing note (example):\r\n- Indexing creates embeddings.\r\n- Queries retrieve relevant chunks as context.\r\n\"\"\",\n    title='sample.txt'\n  )\n)] grounding_supports=[GroundingSupport(\n  grounding_chunk_indices=[\n    0,\n  ],\n  segment=Segment(\n    end_index=117,\n    text='Warm AI is a product with a vision to provide professional networking and research capabilities with grounded answers'\n  )\n), GroundingSupport(\n  grounding_chunk_indic

In [ ]:
# --- Step 4: Beautiful Terminal Display for RAG Results ---
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.markdown import Markdown
from rich.text import Text
from rich.box import ROUNDED

def display_rag_results(query: str, response: Any, latency_ms: int):
    console = Console()
    
    # --- 1.  Input Question Header ---
    console.print("\n")
    console.print(Panel(
        Text(query, style="bold cyan"), 
        title="[bold white]❓ Input Question[/bold white]", 
        border_style="cyan",
        box=ROUNDED,
        padding=(1, 2)
    ))

    # --- 2. Rag Answer Section ----
    console.print(Panel(
        Markdown(response.text), 
        title="[bold white]🤖 Warm AI Search Answer[/bold white]", 
        subtitle=f"[dim]Latency: {latency_ms}ms[/dim]",
        border_style="green",
        box=ROUNDED,
        padding=(1, 2)
    ))

    # ---  3. Grounding & Citations Section ---
    gm = None
    try:
        gm = response.candidates[0].grounding_metadata
    except (AttributeError, IndexError):
        gm = None

    if gm and (getattr(gm, 'grounding_chunks', None) or getattr(gm, 'grounding_supports', None)):
        table = Table(
            title="[bold magenta]📍 Grounding & Citations[/bold magenta]", 
            box=ROUNDED, 
            header_style="bold magenta",
            expand=True,
            show_lines=True
        )
        table.add_column("Source Document", style="bold yellow", ratio=1)
        table.add_column("Evidence / Grounded Segment", style="italic white", ratio=3)
        
        # Access attributes safely (compatible with Pydantic or dict-like objects)
        chunks = getattr(gm, 'grounding_chunks', [])
        supports = getattr(gm, 'grounding_supports', [])
        
        for i, support in enumerate(supports):
            # Extract segment text
            segment = getattr(support, 'segment', None)
            segment_text = getattr(segment, 'text', "N/A").strip()
            
            # Identify source titles/chunks
            indices = getattr(support, 'grounding_chunk_indices', [])
            source_labels = []
            for idx in indices:
                if idx < len(chunks):
                    chunk = chunks[idx]
                    # Check for retrieved_context (typical for File Search)
                    ctx = getattr(chunk, 'retrieved_context', None)
                    title = getattr(ctx, 'title', f"Chunk {idx}")
                    source_labels.append(title)
            
            table.add_row(
                ", ".join(source_labels) if source_labels else "Unknown Source",
                segment_text
            )
            
        console.print(table)
    else:
        console.print(Panel(
            "[italic yellow]No specific grounding citations found for this response.[/italic yellow]", 
            title="[bold yellow]⚠️ No Citations[/bold yellow]",
            border_style="yellow",
            box=ROUNDED
        ))
    console.print("\n")

# --- Run the display ---
display_rag_results(query, resp, ms)

╭─────────────────────────────────────────────── ❓ Input Question ───────────────────────────────────────────────╮
│                                                                                                                 │
│  What is Warm AI and what is planned in v2?                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🤖 Warm AI Search Answer ────────────────────────────────────────────╮
│                                                                                                                 │
│  Warm AI is a product with a vision to provide professional networking and research capabilities with grounded  │
│  answers.                                                                                                       │
│                                                                                                                 │
│  Its key modules include:                                                                                       │
│                                                                                                                 │
│   • Chat mode                                                                                                   │
│   • People search                                                                                               │
│   • Company search                                                                                              │
│   • File Search (RAG) prototype                                                                                 │
│                                                                                                                 │
│  The plans for Roadmap v2 are to:                                                                               │
│                                                                                                                 │
│   1 Add File Search so users can upload a document and chat grounded in it.                                     │
│   2 Extract citations for UI trust.                                                                             │
│                                                                                                                 │
╰──────────────────────────────────────────────── Latency: 2058ms ────────────────────────────────────────────────╯

                                             📍 Grounding & Citations                                              
╭────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────╮
│ Source Document            │ Evidence / Grounded Segment                                                        │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sample.txt                 │ Warm AI is a product with a vision to provide professional networking and research │
│                            │ capabilities with grounded answers                                                 │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sample.txt                 │ Its key modules include:                                                           │
│                            │ *   Chat mode                                                                      │
│                            │ *   People search                                                                  │
│                            │ *   Company search                                                                 │
│                            │ *   File Search (RAG) prototype                                                    │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sample.txt                 │ Add **File Search** so users can upload a document and chat grounded in it         │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sample.txt                 │ Extract **citations** for UI trust                                                 │
╰────────────────────────────┴────────────────────────────────────────────────────────────────────────────────────╯